In [1]:
df = spark.createDataFrame([(1, 4), (2, 5), (3, 6)], ["A", "B"])
df2 = spark.createDataFrame([(1, 4), (2, 5), (3, 6)], ["A", "B"])

In [2]:
display(df)

A,B
1,4
2,5
3,6


In [3]:
df.count()

Out[ 3 ]: 3

In [4]:
df = df.withColumn('C',df.A + df.B)

In [5]:
display(df)

A,B,C
1,4,5
2,5,7
3,6,9


In [6]:
display(df[['A']])

A
1
2
3


In [7]:
from pyspark.sql.functions import lit
df.withColumn('C',lit(5)).show()

+---+---+---+
 A| B| C|
+---+---+---+
 1| 4| 5|
 2| 5| 5|
 3| 6| 5|
+---+---+---+

In [8]:
df.select('*',(df.A > 2).alias("State"),(lit(2)).alias("StateB"),(df.A+df.B).alias("StateC")).show()

+---+---+---+-----+------+------+
 A| B| C|State|StateB|StateC|
+---+---+---+-----+------+------+
 1| 4| 5|false| 2| 5|
 2| 5| 7|false| 2| 7|
 3| 6| 9| true| 2| 9|
+---+---+---+-----+------+------+

In [9]:
from pyspark.sql import functions as F

In [10]:
df.withColumn('N',F.when((df.B > 5) & (df.C >9),20).when(df.A > 2,10).otherwise(1)).show()

+---+---+---+---+
 A| B| C| N|
+---+---+---+---+
 1| 4| 5| 1|
 2| 5| 7| 1|
 3| 6| 9| 10|
+---+---+---+---+

In [11]:
df = spark.createDataFrame([('a',33), ('b',11), ('a',22)],['names','age'])

In [12]:
gb = df.groupBy(['names'])

In [13]:
display(gb.avg('age'))

names,avg(age)
b,11.0
a,27.5


In [14]:
display(gb.count())

names,count
b,1
a,2


In [15]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

In [16]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

In [17]:
def func(x):
  return int(x)+5

strToInt = udf( func, IntegerType())

In [18]:
df = spark.createDataFrame([('1',1),('2',2)],['a','b'])

In [19]:
df.show()

+---+---+
 a| b|
+---+---+
 1| 1|
 2| 2|
+---+---+

In [20]:
df.withColumn('X',strToInt(df.a)).show()

+---+---+---+
 a| b| X|
+---+---+---+
 1| 1| 6|
 2| 2| 7|
+---+---+---+

In [21]:
titanic_data = spark.read.csv('/FileStore/tables/titanic_train-ac800.csv', header=True, inferSchema=True)

In [22]:
def getStrLen(s):
  return len(s)

strLenUDF = udf( getStrLen, IntegerType())

In [23]:
titanic_data.withColumn('NameLen',strLenUDF(titanic_data.Name)).count()

Out[ 56 ]: 891

In [24]:
@pandas_udf('int', PandasUDFType.SCALAR)
def strLenPUDF(c):
  return c.str.len()

titanic_data.withColumn('a',strLenPUDF(titanic_data.Name)).count()

Out[ 59 ]: 891

In [25]:
df = spark.range(100)
sqr = udf(lambda x:x*x, IntegerType())
df = df.withColumn('sqr',df.id)
df = df.withColumn('key',df.sqr%3)
df.show()

+---+---+---+
 id|sqr|key|
+---+---+---+
 0| 0| 0|
 1| 1| 1|
 2| 2| 2|
 3| 3| 0|
 4| 4| 1|
 5| 5| 2|
 6| 6| 0|
 7| 7| 1|
 8| 8| 2|
 9| 9| 0|
 10| 10| 1|
 11| 11| 2|
 12| 12| 0|
 13| 13| 1|
 14| 14| 2|
 15| 15| 0|
 16| 16| 1|
 17| 17| 2|
 18| 18| 0|
 19| 19| 1|
+---+---+---+
only showing top 20 rows

In [26]:
import numpy as np
import pandas as pd

@pandas_udf("key int, sqr int", PandasUDFType.GROUPED_MAP)
def func(pdf):
  print (pdf)
  sqr = pdf.sqr
  r= pdf.assign(sqr=np.mean(pdf.sqr))
  print (r)
  return r

df.select(['key','sqr']).groupBy("key").apply(func).distinct().show()

+---+---+
key|sqr|
+---+---+
 1| 49|
 2| 50|
 0| 49|
+---+---+

In [27]:
import numpy as np

@pandas_udf("key int, sqr int", PandasUDFType.GROUPED_MAP)
def func(pdf):
  print (pdf)
  sqr = pdf.sqr
  return pdf.assign(sqr=(np.sum(pdf.sqr)))

df.select(['key','sqr']).groupBy("key").apply(func).distinct().show()

+---+---------+
key| sqr|
+---+---------+
 0|166167000|
 1|166666500|
+---+---------+

In [28]:
strLenPUDF(titanic_data.Name)

Out[ 69 ]: Column<strLenPUDF(Name)>

In [29]:
@pandas_udf('string', PandasUDFType.SCALAR)
def strLenPUDF(c):
  return c.map(lambda n: "May b Male" if "Mr." in n else "May be Female")

titanic_data.withColumn('FindGender',strLenPUDF(titanic_data.Name)).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------+
PassengerId|Survived|Pclass| Name| Sex| Age|SibSp|Parch| Ticket| Fare|Cabin|Embarked| FindGender|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------+
 1| 0| 3|Braund, Mr. Owen ...| male|22.0| 1| 0| A/5 21171| 7.25| null| S| May b Male|
 2| 1| 1|Cumings, Mrs. Joh...|female|38.0| 1| 0| PC 17599|71.2833| C85| C|May be Female|
 3| 1| 3|Heikkinen, Miss. ...|female|26.0| 0| 0|STON/O2. 3101282| 7.925| null| S|May be Female|
 4| 1| 1|Futrelle, Mrs. Ja...|female|35.0| 1| 0| 113803| 53.1| C123| S|May be Female|
 5| 0| 3|Allen, Mr. Willia...| male|35.0| 0| 0| 373450| 8.05| null| S| May b Male|
 6| 0| 3| Moran, Mr. James| male|null| 0| 0| 330877| 8.4583| null| Q| May b Male|
 7| 0| 1|McCarthy, Mr. Tim...| male|54.0| 0| 0| 17463|51.8625| E46| S| May b Male|
 8| 0| 3|Palsson, Master. ...| male| 2.0| 3| 1| 349909| 21.075| null| S|May be Female|
 9| 1| 3|Johnson, Mrs. Osc...|female|27.0| 0| 2| 347742|11.1333| null| S|May be Female|
 10| 1| 2|Nasser, Mrs. Nich...|female|14.0| 1| 0| 237736|30.0708| null| C|May be Female|
 11| 1| 3|Sandstrom, Miss. ...|female| 4.0| 1| 1| PP 9549| 16.7| G6| S|May be Female|
 12| 1| 1|Bonnell, Miss. El...|female|58.0| 0| 0| 113783| 26.55| C103| S|May be Female|
 13| 0| 3|Saundercock, Mr. ...| male|20.0| 0| 0| A/5. 2151| 8.05| null| S| May b Male|
 14| 0| 3|Andersson, Mr. An...| male|39.0| 1| 5| 347082| 31.275| null| S| May b Male|
 15| 0| 3|Vestrom, Miss. Hu...|female|14.0| 0| 0| 350406| 7.8542| null| S|May be Female|
 16| 1| 2|Hewlett, Mrs. (Ma...|female|55.0| 0| 0| 248706| 16.0| null| S|May be Female|
 17| 0| 3|Rice, Master. Eugene| male| 2.0| 4| 1| 382652| 29.125| null| Q|May be Female|
 18| 1| 2|Williams, Mr. Cha...| male|null| 0| 0| 244373| 13.0| null| S| May b Male|
 19| 0| 3|Vander Planke, Mr...|female|31.0| 1| 0| 345763| 18.0| null| S|May be Female|
 20| 1| 3|Masselmani, Mrs. ...|female|null| 0| 0| 2649| 7.225| null| C|May be Female|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------------+
only showing top 20 rows

In [30]:
stringJSONRDD = sc.parallelize((""" 
  { "id": "123",
    "name": "Katie",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Michael",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Simone",
    "age": 23,
    "eyeColor": "blue"
  }""")
)

In [31]:
swimmer = spark.read.json(stringJSONRDD)

In [32]:
swimmer.createOrReplaceTempView("s")

In [33]:
spark.sql("select * from s limit 2").show()

+---+--------+---+-------+
age|eyeColor| id| name|
+---+--------+---+-------+
 19| brown|123| Katie|
 22| green|234|Michael|
+---+--------+---+-------+

In [34]:
swimmer = swimmer.withColumn('age',lit(10))

In [35]:
swimmer.show()

+---+--------+---+-------+
age|eyeColor| id| name|
+---+--------+---+-------+
 10| brown|123| Katie|
 10| green|234|Michael|
 10| blue|345| Simone|
+---+--------+---+-------+

In [36]:
spark.sql("select * from s where age > 20").show()

+---+--------+---+-------+
age|eyeColor| id| name|
+---+--------+---+-------+
 22| green|234|Michael|
 23| blue|345| Simone|
+---+--------+---+-------+

In [37]:
spark.catalog.listTables()

Out[ 116 ]: 
[Table(name=u'cs_test_31b97_csv', database=u'default', description=None, tableType=u'EXTERNAL', isTemporary=False),
 Table(name=u'hr_comma_sep_csv_55265_txt', database=u'default', description=None, tableType=u'EXTERNAL', isTemporary=False),
 Table(name=u'hr_data', database=u'default', description=None, tableType=u'EXTERNAL', isTemporary=False),
 Table(name=u's', database=None, description=None, tableType=u'TEMPORARY', isTemporary=True),
 Table(name=u'swiimerview', database=None, description=None, tableType=u'TEMPORARY', isTemporary=True),
 Table(name=u'swimmerview', database=None, description=None, tableType=u'TEMPORARY', isTemporary=True)]

In [38]:
from pyspark.sql.types import *

stringCSVRDD = sc.parallelize([(123, 'Katie', 19, 'brown'), (234, 'Michael', 22, 'green'), (345, 'Simone', 22, 11)])

In [39]:
from pyspark.sql.types import StringType, StructField, LongType, StructType

In [40]:
schemaString = "id name age eyeColor"

In [41]:
schema = StructType([
    StructField("id", LongType(), True),    
    StructField("name", StringType(), True),
    StructField("age", LongType(), True),
    StructField("eyeColor", StringType(), True)
])

In [42]:
swimmers = spark.createDataFrame(stringCSVRDD, schema)

In [43]:
swimmers.write.csv("a.csv")

In [44]:
from pyspark.sql.types import Row

In [45]:
# Create the Departments
department1 = Row(id='123456', name='Computer Science')
department2 = Row(id='789012', name='Mechanical Engineering')
department3 = Row(id='345678', name='Theater and Drama')
department4 = Row(id='901234', name='Indoor Recreation')

In [46]:
Employee = Row("firstName", "lastName", "email", "salary")

In [47]:
employee1 = Employee('michael', 'armbrust', 'no-reply@berkeley.edu', 100000)
employee11 = Employee('michael', 'armbrust', 'no-reply@berkeley.edu', 200000)
employee2 = Employee('xiangrui', 'meng', 'no-reply@stanford.edu', 120000)
employee3 = Employee('matei', None, 'no-reply@waterloo.edu', 140000)
employee31 = Employee('matei', None, 'no-reply@waterloo.edu', 180000)
employee4 = Employee(None, 'wendell', 'no-reply@berkeley.edu', 160000)

In [48]:
departmentWithEmployees1 = Row(department=department1, employees=[employee1, employee2])
departmentWithEmployees2 = Row(department=department2, employees=[employee3, employee4, employee11])
departmentWithEmployees3 = Row(department=department3, employees=[employee1, employee4, employee31])
departmentWithEmployees4 = Row(department=department4, employees=[employee2, employee3])

In [49]:
df = spark.createDataFrame([departmentWithEmployees1,departmentWithEmployees2,departmentWithEmployees3,departmentWithEmployees4])

In [50]:
display(df)

department,employees
"List(123456, Computer Science)","List(List(michael, armbrust, no-reply@berkeley.edu, 100000), List(xiangrui, meng, no-reply@stanford.edu, 120000))"
"List(789012, Mechanical Engineering)","List(List(matei, null, no-reply@waterloo.edu, 140000), List(null, wendell, no-reply@berkeley.edu, 160000), List(michael, armbrust, no-reply@berkeley.edu, 200000))"
"List(345678, Theater and Drama)","List(List(michael, armbrust, no-reply@berkeley.edu, 100000), List(null, wendell, no-reply@berkeley.edu, 160000), List(matei, null, no-reply@waterloo.edu, 180000))"
"List(901234, Indoor Recreation)","List(List(xiangrui, meng, no-reply@stanford.edu, 120000), List(matei, null, no-reply@waterloo.edu, 140000))"


In [51]:
from pyspark.sql.functions import explode
df = df.select("*", explode("employees").alias("e"))

In [52]:
display(df)

department,employees,e
"List(123456, Computer Science)","List(List(michael, armbrust, no-reply@berkeley.edu, 100000), List(xiangrui, meng, no-reply@stanford.edu, 120000))","List(michael, armbrust, no-reply@berkeley.edu, 100000)"
"List(123456, Computer Science)","List(List(michael, armbrust, no-reply@berkeley.edu, 100000), List(xiangrui, meng, no-reply@stanford.edu, 120000))","List(xiangrui, meng, no-reply@stanford.edu, 120000)"
"List(789012, Mechanical Engineering)","List(List(matei, null, no-reply@waterloo.edu, 140000), List(null, wendell, no-reply@berkeley.edu, 160000), List(michael, armbrust, no-reply@berkeley.edu, 200000))","List(matei, null, no-reply@waterloo.edu, 140000)"
"List(789012, Mechanical Engineering)","List(List(matei, null, no-reply@waterloo.edu, 140000), List(null, wendell, no-reply@berkeley.edu, 160000), List(michael, armbrust, no-reply@berkeley.edu, 200000))","List(null, wendell, no-reply@berkeley.edu, 160000)"
"List(789012, Mechanical Engineering)","List(List(matei, null, no-reply@waterloo.edu, 140000), List(null, wendell, no-reply@berkeley.edu, 160000), List(michael, armbrust, no-reply@berkeley.edu, 200000))","List(michael, armbrust, no-reply@berkeley.edu, 200000)"
"List(345678, Theater and Drama)","List(List(michael, armbrust, no-reply@berkeley.edu, 100000), List(null, wendell, no-reply@berkeley.edu, 160000), List(matei, null, no-reply@waterloo.edu, 180000))","List(michael, armbrust, no-reply@berkeley.edu, 100000)"
"List(345678, Theater and Drama)","List(List(michael, armbrust, no-reply@berkeley.edu, 100000), List(null, wendell, no-reply@berkeley.edu, 160000), List(matei, null, no-reply@waterloo.edu, 180000))","List(null, wendell, no-reply@berkeley.edu, 160000)"
"List(345678, Theater and Drama)","List(List(michael, armbrust, no-reply@berkeley.edu, 100000), List(null, wendell, no-reply@berkeley.edu, 160000), List(matei, null, no-reply@waterloo.edu, 180000))","List(matei, null, no-reply@waterloo.edu, 180000)"
"List(901234, Indoor Recreation)","List(List(xiangrui, meng, no-reply@stanford.edu, 120000), List(matei, null, no-reply@waterloo.edu, 140000))","List(xiangrui, meng, no-reply@stanford.edu, 120000)"
"List(901234, Indoor Recreation)","List(List(xiangrui, meng, no-reply@stanford.edu, 120000), List(matei, null, no-reply@waterloo.edu, 140000))","List(matei, null, no-reply@waterloo.edu, 140000)"


In [53]:
df.selectExpr("department.id","department.name","e.firstname","e.email","e.salary").show()

+------+--------------------+---------+--------------------+------+
 id| name|firstname| email|salary|
+------+--------------------+---------+--------------------+------+
123456| Computer Science| michael|no-reply@berkeley...|100000|
123456| Computer Science| xiangrui|no-reply@stanford...|120000|
789012|Mechanical Engine...| matei|no-reply@waterloo...|140000|
789012|Mechanical Engine...| null|no-reply@berkeley...|160000|
789012|Mechanical Engine...| michael|no-reply@berkeley...|200000|
345678| Theater and Drama| michael|no-reply@berkeley...|100000|
345678| Theater and Drama| null|no-reply@berkeley...|160000|
345678| Theater and Drama| matei|no-reply@waterloo...|180000|
901234| Indoor Recreation| xiangrui|no-reply@stanford...|120000|
901234| Indoor Recreation| matei|no-reply@waterloo...|140000|
+------+--------------------+---------+--------------------+------+

In [54]:
sjr = sc.parallelize((""" 
  { "id": "{'h':"234"}",
    "eyeColor": ["brown","yellow"]
  }""",
   """{
    "id": {'h':"234"}""
    "eyeColor": ["green","purple"]
  }""", 
  """{
    "id": {'g':"234"},
    "eyeColor": ["blue"]
  }""")
)

In [55]:
df = spark.read.json(sjr)

In [56]:
df.select("*",explode(df.eyeColor).alias("c")).show()

+---------------+-------+------+
 eyeColor| id| c|
+---------------+-------+------+
[brown, yellow]|[, 234]| brown|
[brown, yellow]|[, 234]|yellow|
[green, purple]|[, 234]| green|
[green, purple]|[, 234]|purple|
 [blue]| [234,]| blue|
+---------------+-------+------+

In [57]:
data = spark.read.json('/FileStore/tables/mixed.json')

In [58]:
data.selectExpr("address.zip").show()

+-----+
 zip|
+-----+
 null|
98016|
 null|
+-----+

In [59]:
display(data)

a,address,id,orders,since
1.0,null,101,"List(List(null, 100001))",2001
2.0,List(98016),102,null,1999
3.1,null,103,"List(List(B001, 300001), List(B005, 300002), List(A007, 300003))",foo


In [60]:
data_ex = data.select("*",explode("orders").alias("o")).show()

+---+-------+---+--------------------+-----+--------------+
 a|address| id| orders|since| o|
+---+-------+---+--------------------+-----+--------------+
1.0| null|101| [[, 100001]]| 2001| [, 100001]|
3.1| null|103|[[B001, 300001], ...| foo|[B001, 300001]|
3.1| null|103|[[B001, 300001], ...| foo|[B005, 300002]|
3.1| null|103|[[B001, 300001], ...| foo|[A007, 300003]|
+---+-------+---+--------------------+-----+--------------+

In [61]:
display(data)

a,address,id,orders,since
1.0,null,101,"List(List(null, 100001))",2001
2.0,List(98016),102,null,1999
3.1,null,103,"List(List(B001, 300001), List(B005, 300002), List(A007, 300003))",foo


In [62]:
data.select("id",explode("orders")).show()

+---+--------------+
 id| col|
+---+--------------+
101| [, 100001]|
103|[B001, 300001]|
103|[B005, 300002]|
103|[A007, 300003]|
+---+--------------+

In [63]:
rdd = sc.textFile('/FileStore/tables/movie_json-7ce1e.txt')

In [64]:
rdd.filter(lambda x: len(x) != 0).map(lambda x: x.strip()).collect()

Out[ 193 ]: 
[u'{',
 u'"David Smith":',
 u'{',
 u'"Vertigo": 4,',
 u'"Scarface": 4.5,',
 u'"Raging Bull": 3.0,',
 u'"Goodfellas": 4.5,',
 u'"The Apartment": 1.0',
 u'},',
 u'"Brenda Peterson":',
 u'{',
 u'"Vertigo": 3.0,',
 u'"Scarface": 1.5,',
 u'"Raging Bull": 1.0,',
 u'"Goodfellas": 2.0,',
 u'"The Apartment": 5.0,',
 u'"Roman Holiday": 4.5',
 u'},',
 u'"Bill Duffy":',
 u'{',
 u'"Vertigo": 4.5,',
 u'"Scarface": 5.0,',
 u'"Goodfellas": 4.5,',
 u'"The Apartment": 1.0',
 u'},',
 u'"Samuel Miller":',
 u'{',
 u'"Scarface": 3.5,',
 u'"Raging Bull": 5.0,',
 u'"The Apartment": 1.0,',
 u'"Goodfellas": 5.0,',
 u'"Roman Holiday": 1.0',
 u'},',
 u'"Julie Hammel":',
 u'{',
 u'"Scarface": 2.5,',
 u'"Roman Holiday": 4.5,',
 u'"Goodfellas": 3.0',
 u'},',
 u'"Clarissa Jackson":',
 u'{',
 u'"Vertigo": 5.0,',
 u'"Scarface": 4.5,',
 u'"Raging Bull": 4.0,',
 u'"Goodfellas": 2.5,',
 u'"The Apartment": 1.0,',
 u'"Roman Holiday": 1.5',
 u'},',
 u'"Adam Cohen":',
 u'{',
 u'"Vertigo": 3.5,',
 u'"Scarface": 3.0,',
 u'"The Apartment": 1.0,',
 u'"Goodfellas": 4.5,',
 u'"Roman Holiday": 3.0',
 u'},',
 u'"Chris Duncan":',
 u'{',
 u'"The Apartment": 1.5,',
 u'"Raging Bull": 4.5',
 u'}',
 u'}']

In [65]:
sc.textFile('/FileStore/tables/movie_json-7ce1e.txt').collect()

Out[ 195 ]: 
[u'{',
 u' { "Vertigo": 4, "Scarface": 4.5, "Raging Bull": 3.0, "Goodfellas": 4.5, "The Apartment": 1.0 },',
 u' { "Vertigo": 3.0, "Scarface": 1.5, "Raging Bull": 1.0, "Goodfellas": 2.0, "The Apartment": 5.0, "Roman Holiday": 4.5 },',
 u'}']

In [66]:
df = spark.read.json('/FileStore/tables/movie_json-7ce1e.txt')

In [68]:
df.show()

+----------+-----------+-------------+--------+-------------+-------+---------------+
Goodfellas|Raging Bull|Roman Holiday|Scarface|The Apartment|Vertigo|_corrupt_record|
+----------+-----------+-------------+--------+-------------+-------+---------------+
 null| null| null| null| null| null| {|
 4.5| 3.0| null| 4.5| 1.0| 4.0| null|
 2.0| 1.0| 4.5| 1.5| 5.0| 3.0| null|
 null| null| null| null| null| null| }|
+----------+-----------+-------------+--------+-------------+-------+---------------+

In [69]:
spark.read.json('/FileStore/tables/movie_json-7ce1e.txt').show()

+----------+-----------+-------------+--------+-------------+-------+
Goodfellas|Raging Bull|Roman Holiday|Scarface|The Apartment|Vertigo|
+----------+-----------+-------------+--------+-------------+-------+
 4.5| 3.0| null| 4.5| 1.0| [4, 4]|
 2.0| 1.0| 4.5| 1.5| 5.0| [1, 2]|
+----------+-----------+-------------+--------+-------------+-------+